In [8]:
import pyspark
import findspark
findspark.find()
findspark.init()
from pyspark.sql import *
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark import SparkConf
from pyspark import SparkContext
sc=SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))


In [27]:
#machine learning libs......

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.types import *
from pyspark.sql.functions import UserDefinedFunction
from IPython import get_ipython
from pyspark.ml.feature import HashingTF,Tokenizer

In [31]:
sqlcontext=SQLContext(sc) 

In [12]:
input_file_path="file:///C:/Users/ckp43_000/Documents/food-inspections.csv"
food_df=sqlcontext.read.csv(input_file_path,inferSchema=True,header=True)

In [13]:
food_df.show(10)

+-------------+--------------------+--------------------+---------+--------------------+-------------+--------------------+-------+-----+-----+--------------------+--------------------+------------------+--------------------+------------------+------------------+--------------------+--------------------------+---------+---------------+-------------+-----+
|Inspection ID|            DBA Name|            AKA Name|License #|       Facility Type|         Risk|             Address|   City|State|  Zip|     Inspection Date|     Inspection Type|           Results|          Violations|          Latitude|         Longitude|            Location|Historical Wards 2003-2015|Zip Codes|Community Areas|Census Tracts|Wards|
+-------------+--------------------+--------------------+---------+--------------------+-------------+--------------------+-------+-----+-----+--------------------+--------------------+------------------+--------------------+------------------+------------------+--------------------+

In [14]:
food_df.printSchema()

root
 |-- Inspection ID: integer (nullable = true)
 |-- DBA Name: string (nullable = true)
 |-- AKA Name: string (nullable = true)
 |-- License #: integer (nullable = true)
 |-- Facility Type: string (nullable = true)
 |-- Risk: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zip: integer (nullable = true)
 |-- Inspection Date: string (nullable = true)
 |-- Inspection Type: string (nullable = true)
 |-- Results: string (nullable = true)
 |-- Violations: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- Historical Wards 2003-2015: string (nullable = true)
 |-- Zip Codes: string (nullable = true)
 |-- Community Areas: string (nullable = true)
 |-- Census Tracts: string (nullable = true)
 |-- Wards: string (nullable = true)



In [15]:
food_df.show(5)

+-------------+--------------------+--------------------+---------+--------------------+-------------+--------------------+-------+-----+-----+--------------------+--------------------+------------------+--------------------+------------------+------------------+--------------------+--------------------------+---------+---------------+-------------+-----+
|Inspection ID|            DBA Name|            AKA Name|License #|       Facility Type|         Risk|             Address|   City|State|  Zip|     Inspection Date|     Inspection Type|           Results|          Violations|          Latitude|         Longitude|            Location|Historical Wards 2003-2015|Zip Codes|Community Areas|Census Tracts|Wards|
+-------------+--------------------+--------------------+---------+--------------------+-------------+--------------------+-------+-----+-----+--------------------+--------------------+------------------+--------------------+------------------+------------------+--------------------+

In [37]:
food_Table=food_df.createOrReplaceTempView("food_Table")

In [38]:
sqlcontext.sql("""
describe table food_Table""").show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|       Inspection ID|      int|   null|
|            DBA Name|   string|   null|
|            AKA Name|   string|   null|
|           License #|      int|   null|
|       Facility Type|   string|   null|
|                Risk|   string|   null|
|             Address|   string|   null|
|                City|   string|   null|
|               State|   string|   null|
|                 Zip|      int|   null|
|     Inspection Date|   string|   null|
|     Inspection Type|   string|   null|
|             Results|   string|   null|
|          Violations|   string|   null|
|            Latitude|   double|   null|
|           Longitude|   double|   null|
|            Location|   string|   null|
|Historical Wards ...|   string|   null|
|           Zip Codes|   string|   null|
|     Community Areas|   string|   null|
+--------------------+---------+-------+
only showing top

In [40]:
sqlcontext.sql("""
select b* from food_Table limit 10""").show()

+-------------+--------------------+--------------------+---------+--------------------+-------------+--------------------+-------+-----+-----+--------------------+--------------------+------------------+--------------------+------------------+------------------+--------------------+--------------------------+---------+---------------+-------------+-----+
|Inspection ID|            DBA Name|            AKA Name|License #|       Facility Type|         Risk|             Address|   City|State|  Zip|     Inspection Date|     Inspection Type|           Results|          Violations|          Latitude|         Longitude|            Location|Historical Wards 2003-2015|Zip Codes|Community Areas|Census Tracts|Wards|
+-------------+--------------------+--------------------+---------+--------------------+-------------+--------------------+-------+-----+-----+--------------------+--------------------+------------------+--------------------+------------------+------------------+--------------------+

In [41]:
sqlcontext.sql("""
select count(*) from food_Table""").show()

+--------+
|count(1)|
+--------+
|  196825|
+--------+



### Creating rdd from a file
     --->>> to create rdd we need SparkContext (sc)
          rdd=sc.textFile('file_path')

In [32]:
food_rdd=sc.textFile(input_file_path)

In [33]:
food_rdd.take(1) 

['Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards']

In [34]:
food_rdd.take(3)

['Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards',
 '2352683,CHIPOTLE MEXICAN GRILL,CHIPOTLE MEXICAN GRILL,2670642,Restaurant,Risk 1 (High),1025 W ADDISON ST ,CHICAGO,IL,60613,2019-12-04T00:00:00.000,License Re-Inspection,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL EMPLOYEE; KNOWLEDGE, RESPONSIBILITIES AND REPORTING - Comments: FOUND NO EMPLOYEE HEALTH POLICY AT PREMISES. PRIORITY FOUNDATION. NO CITATION ISSUED. 7-38-010. MUST PROVIDE.",41.94711774835235,-87.6553811622124,"{\'latitude\': \'-87.6553811622124\', \'longitude\': \'41.94711774835235\'}",,,,,',
 '2352734,CHILI\'S T-I,CHILI\'S (T1-B14),34169,Restaurant,Risk 1 (High),11601 W TOUHY AVE ,CHICAGO,IL,60666,2019-12-04T00:00:00.000,Canvass,Pass,"10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLIED AND ACCESSIBLE - Comments: OBSERV